In [1]:
import os, json

with open('conf.json', 'r') as f:
        json_data = json.load(f)
    
os.environ['OPENAI_API_KEY'] = json_data['API_KEY']

In [2]:
from core.get_common_issues import get_news_dict_list

etf_tkr = "AIQ"

news_data = str(get_news_dict_list(etf_tkr=etf_tkr))

In [6]:
from data.fetch_data import fetch_data_from_db

import pandas as pd

sql = """
    select etf_tkr from os_index_info
"""

etf_tkrs = pd.DataFrame(fetch_data_from_db(query=sql)).etf_tkr.to_list()

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI

from tqdm.auto import tqdm

def get_llm_answer(text):
    template = ChatPromptTemplate.from_messages(
        [
            SystemMessage(
                content=(
                    "You are now acting as an intelligent fund manager fluent in both English and Korean. I will provide you with news headlines of different stock tickers. Your task is to identify common themes that appear across all these different tickers, not just within each individual one. Please summarize these overarching themes in Korean, in a coherent format limited to 10 sentences. Ensure your response is tailored to be informative for stock investors and is presented in a professional manner. Additionally, please conclude your answer with a noun. don't answer in bullet points"
                )
            ),
            HumanMessagePromptTemplate.from_template("{text}"),
        ]
    )

    llm = ChatOpenAI(model="gpt-3.5-turbo")
    
    ret = llm(template.format_messages(text=text))

    return ret

for etf_tkr in tqdm(etf_tkrs):
    
    news_data_per_etf = str(get_news_dict_list(etf_tkr=etf_tkr))

    llm_answer = get_llm_answer(text=news_data_per_etf)

    print(llm_answer)
    print()

    with open(f'./etf_llm_summary/{etf_tkr}.json', 'w') as outfile:
        json.dump(llm_answer, outfile, indent='\t', ensure_ascii=False)

    break

  0%|          | 0/34 [00:00<?, ?it/s]

AIMessage(content='한국어 번역:\n\n비록 다양한 회사의 주식이지만, 주식 투자자들에게 공통적으로 주목해야 할 테마들이 있습니다. 제공된 뉴스 헤드라인을 통해 이러한 주요 테마들을 파악할 수 있습니다.\n\n첫째, 보험 및 의료 부문에서의 미국 정부의 의약품 가격 협상은 주목할 가치가 있습니다. 메디케어 가격 협상 목록에 의해 의료 기업들이 영향을 받을 수 있음을 알 수 있습니다. 이는 주식 투자에 영향을 미칠 수 있는 중요한 요소입니다.\n\n둘째, 암 치료 부문은 큰 잠재력을 가지고 있으며, 관련 주식들에 주목할 가치가 있습니다. 이러한 회사들은 의료 부문에서 큰 관심을 받고 있으며, 투자 기회를 제공할 수 있습니다.\n\n셋째, 메디케어의 10가지 약물 목록은 몇몇 의약품 기업들이 무시하고 있음을 알 수 있습니다. 그러나 이 목록이 앞으로 의약품 기업들에게 어떤 영향을 미칠지에 대한 관심은 계속해야 합니다.\n\n넷째, 의약품 기업들은 비용이 많이 드는 약물들이 미국에서 처음으로 가격 협상의 대상이 될 수 있다는 점에 대해 주목해야 합니다. 이러한 협상은 의약품 기업들에게 중요한 영향을 미칠 수 있으며, 이를 고려하여 투자 결정을 해야 합니다.\n\n다섯째, 의약품 기업들의 실적과 분석 보고서도 주목할 가치가 있습니다. 이는 투자자들이 향후 주식 가치를 예측하고 투자를 결정하는 데 도움이 될 수 있습니다.\n\n이러한 테마들은 다양한 주식들 사이에서 공통적으로 등장하는 것으로 보입니다. 이러한 정보들을 종합하여 투자 결정에 활용할 수 있습니다.\n\n주식 투자자들에게 유용한 정보를 제공하기 위해, 주식에 대한 종합적인 주제를 다음과 같이 요약합니다.\n\n"미국 정부의 의약품 가격 협상 정책, 암 치료 부문의 잠재력, 메디케어 약물 가격 협상 목록에 대한 관심, 비용이 많이 드는 약물들의 협상 대상, 실적과 분석 보고서 등의 주요 테마를 고려하여 투자 결정을 해야 합니다."\n\n주제(noun): 투자 결정', additional_kwar

  0%|          | 0/34 [00:20<?, ?it/s]
